## Softmax Regression (Multinomial Logistic Regression)

It is the convention in programming that the first thing you do is print "Hello World".
So, like programming has Hello World, machine learning has MNIST.

### What is MNIST?
MNIST is a standard dataset used for computer vision. It consists of images of handwritten digits like the following:

![Figure 1. Sample images from MNIST dataset (from [TensorFlow](https://www.tensorflow.org/get_started/mnist/beginners).](figures/MNIST.png)
Along with the images are the labels for each of them, indicating which digit it is. For instance, the labels for the above images are 5, 0, 4, and 1.

For this tutorial, we are going to implement a simple model and train it to look at images, then predict what their labels are. However, take note that the model to be implemented here will not achieve state-of-the-art performance. We'll get to that later, at the next tutorial. For now, we shall be starting with a quite simple model called the ***Softmax Regression***.

We shall accomplish the following in this tutorial:
* Learn about the MNIST data and softmax regression
* Implement a model for recognizing MNIST digits, based on looking at every pixel of each image.
* Use TensorFlow to train the model to recognize the handwritten digits by having it "look" at thousands of examples
* Check the model's accuracy with the test data

### The MNIST Data
The MNIST data is hosted on [Yann LeCun's website](http://yann.lecun.com/exdb/mnist/). We shall be loading the dataset using the following two lines of code:

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/home/darth/MNIST_data', one_hot=True)

Extracting /home/darth/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/darth/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/darth/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/darth/MNIST_data/t10k-labels-idx1-ubyte.gz


The MNIST data is split into two parts:

|Filename|Category|File Size|
|--------|--------|---------|
|[train-images-idx3-ubyte.gz](http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz)|training set images|9912422 bytes|
|[train-labels-idx1-ubyte.gz](http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz)|training set labels|28881 bytes|
|[t10k-images-idx3-ubyte.gz](http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz)|test set images|1648877 bytes|
|[t10k-labels-idx1-ubyte.gz](http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz)|test set labels|4542 bytes|

The splitting of data is quite important for it is essential in machin learning to have a separate data. This way, we can determine if the model actually generalizes, and not just memorized the data.

As it was mentioned a while ago, the dataset has two parts: (1) image of handwritten digit -- we'll call `x`, and (2) a corresponding label -- we'll call `y`. Both the training set and test set contain images and their corresponding labels, e.g. `x = mnist.train.images`, and `y = mnist.train.labels`.

Each image is 28 pixels by 28 pixels, and they can be interpreted as a big array of numbers:

![](figures/MNIST-Matrix.png)

The said array can then be flatten into a vector of 28x28 = 784 numbers. From this perspective, the MNIST images are just points in a 784-dimensional vector space.

Hence, the result of a flattened `mnist.train.images` is a tensor (n-dimensional array) with a shape of `[55000, 784]`. The first dimension refers to the index of images in the dataset, while the second dimension refers to the index for each pixel in each image. Each entry in the tensor is a pixel intensity between 0 and 1, for a particular pixel in a particular image.

![](figures/mnist-train-xs.png)

As mentioned before, each image in MNIST has a corresponding label, a number between 0 and 9, representing the digit written in the image.

For this tutorial, the labels will be _one-hot vectors_, i.e. a vector with 1 in a single dimension (index of the label for the image) and 0 in the rest. For instance, a label 3 would be `[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]`. Consequently, `mnist.train.labels` is a `[55000, 10]` array of floats.

![](figures/mnist-train-ys.png)

Now that you have been familiarized with the MNIST dataset, we can now write the model.

### Softmax Regression (Multinomial Logistic Regression)

What is **regression**? It is an approximation technique used to find or to estimate the relationships between or among variables. In other words, to determine the mapping of the input and output: \\(f(x) = y\\).

There are of regression techniques available, but for this tutorial, we are going to focus on _multinomial logistic regression_ or more commonly known as _softmax regression_.

Since it has already been understood that every image in MNIST is of a handwritten digit between 0 and 9, there are only 10 possible numbers a given image can be. Through softmax regression, we can look at an image and provide probabilities for each digit. For instance, the model might look at a picture of number 9, and give a probability that says it's 90% sure it's a 9, but give 5% chance to it being an 8, and scattered probabilities among others because it isn't 100% sure.

Hence, if the problem domain is to determine the probabilities of an object belonging to one of several different things (classes), softmax is the one to choose. This is so because it gives a list of values between 0 and 1, that add up to 1 (a whole probability). Even when you look at other models, the most common final step is a softmax layer.

A softmax regression has two steps:
* Add up the evidence of the input being in a certain class
* Convert the evidence to probabilities

To add the evidences that a given image belongs to a particular class, we perform weighted sum of pixel intensities (the input variable _x_).

![](figures/softmax-regression-scalargraph.png)

<br><br>
Converting the diagram to a set of equations,

![](figures/softmax-regression-scalarequation.png)

In form of matrix multiplication and vector addition, the above equations will become the following:

![](figures/softmax-regression-vectorequation.png)

Or in a more compact form,

\\[y = softmax(Wx+b) \\]

Formalizing,

\\[evidence_{i} = \sum_{j} W_{i,j}x_{j} + b_{i}\\]

where \\(W_{i}\\) is the weights, \\(b_{i}\\) is the bias for class \\(i\\), and \\(j\\) is an index for summing over the pixels in the input image \\(x\\). The evidence tallies shall then be converted to predicted probabilities \\(y\\) using the "softmax" function:

\\[y = softmax(evidence)\\]

Expanding the equation,

\\[softmax(x)_{i} = \dfrac{exp(x_{i})}{\sum_{j}exp(x_{j})   }\\]

With the above equation, no hypothesis will ever get zero nor a negative weight. Softmax normalizes the weights, so that they add up to 1, forming a valid probability distribution.

### Implementing the Regression

The machine intelligence library to be used for implementing the regression model will be TensorFlow, and to use it, we must import it.

In [2]:
import tensorflow as tf

In TensorFlow, we describe a graph of operations, operations such as matrix multiplication, and vector addition among others. Such operations are described using symbolic variables like the following:

In [3]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 784])

Unlike in conventional programming, `x` is not a specific value, it's a `placeholder`. An object to hold the value we shall input when we ask TensorFlow to run a computation. For the MNIST case, we want to be able to input any number of images, each flattened into a 784-dimensional vector. We represent this as a 2-D tensor of floating-point numbers, with a shape of `[None, 784]`. `None` was used to denote that the dimension can be of variable length.

We will also need the weights and biases for our model. The weights and biases are the parameters that the model learns through training iterations. Instead of using `placeholder`, we use `Variable` for the parameters so that it is modifiable throughout its lifetime in a TensorFlow graph.

In [4]:
weights = tf.Variable(initial_value=tf.zeros([784, 10]))
biases = tf.Variable(initial_value=tf.zeros([10]))

The `Variable`s are created by initializing them with zeros since they are going to be learned. Hence, it is not of concern what they initially are.

Take note that `weights` has a shape of `[784, 10]`. This is because we want to multiply the 784-dimensional image vectors by it to produce 10-dimensional vectors, consisting of evidence for different classes. Meanwhile, `biases` has a shape of `[10]` so we can add it to the output of the matrix multiplication, i.e. \\((Wx)\\) to be \\((Wx) + b\\).

To implement the model,

In [5]:
# y = tf.nn.softmax(tf.matmul(x, weights) + biases)
y_ = tf.matmul(x, weights) + biases

### Training the Model

To train our model, we must determine how to say if the model is good. Though in ML, we actually measure how bad a model is. That measure is called _loss_ or _cost_. It indicates how far off the predicted output is from the actual output. During the training, we try to minimize that error. The smaller the error, of course, the better.

The conventional function used for computing loss is the _cross entropy_ (from information theory):

\\[H_{y'}(y) = - \sum_{i} y'_{i} log(y_{i})\\]

where $y'$ is the true output, and $y$ is the predicted output.

To implement cross-entropy, we must first define a new placeholder to input the correct answers:

In [6]:
y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

Then, we can implement the cross-entropy function:

In [9]:
# cross entropy function based on formula
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# we don't use the above code since that is numerically unstable, instead we use
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y))

We can now apply the optimization algorithm of our choice:

In [10]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In this case, we use SGD to minimize the loss of our model, with a learning rate of `0.5`.

Let us now launch the model.

In [11]:
# create a TensorFlow session
sess = tf.InteractiveSession()

In [12]:
# initialize the variables
init_op = tf.global_variables_initializer()

sess.run(init_op)

In [13]:
# train 20000 times
for step in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    _, loss = sess.run([train_step, cross_entropy], feed_dict={x: batch_xs, y: batch_ys})
    if step % 100 == 0:
        print('Step loss : {}'.format(loss))

Step loss : 2.3025848865509033
Step loss : 0.4590816795825958
Step loss : 0.2645539343357086
Step loss : 0.41398847103118896
Step loss : 0.3004906177520752
Step loss : 0.1506330370903015
Step loss : 0.2550148069858551
Step loss : 0.3431825637817383
Step loss : 0.315665602684021
Step loss : 0.28851184248924255


### Evaluating our Model

Since we are dealing with one-hot vectors, we must determine which index is of the highest entry in a tensor along an axis. We can accomplish this using `tf.argmax()`, then we check if the labels (predicted and actual) are a match using `tf.equal()`.

In [14]:
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))

In [15]:
# the preceding code gives us a list of booleans,
# we should cast them to floating-point numbers,
# then get its mean.
# For example [True, False, True, True] would become
# [1, 0, 1, 1] which would be 0.75

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [16]:
# finally, compute the accuracy on test data
feed_dict={x: mnist.test.images, y: mnist.test.labels}
print(sess.run(accuracy, feed_dict=feed_dict))

0.9164


An accuracy of $\approx$92% for the simple Softmax Regression model. Is that good? Nope! It can be as high as 98-99% with Convolutional Neural Networks! 